In [4]:
from nytimesarticle import articleAPI
api = articleAPI('tcrAZ92puwt0ApwLWpHkLjS9JIYb35SZ')

from bs4 import BeautifulSoup
from urllib.request import *
import re 
import string

# import NLTK package
from nltk import word_tokenize
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer, PorterStemmer
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import WordNetLemmatizer

# import SpaCy package
import en_core_web_sm
nlp = en_core_web_sm.load()
from spacy.tokenizer import Tokenizer
tokenizer = Tokenizer(nlp.vocab)

In [8]:
articles = api.search( q = 'President Xi' )

In [9]:
mylink = []
print('\n--1--\n')
    
for element in articles:
    print(element)

print('\n--2--\n')
    
for element in articles['response']:
    print(element)

print('\n--3--\n')
    
for element in articles['response']['docs'][0]:
    print(element)

print('\n--4--\n')

print(articles['response']['docs'][0]['web_url'])
print(articles['response']['docs'][0]['snippet'])

print('\n--5--\n')

for element in articles['response']['docs']:
    mylink.append(element['web_url'])
    print(element['web_url'])


--1--

status
copyright
response

--2--

docs
meta

--3--

web_url
snippet
lead_paragraph
blog
source
multimedia
headline
keywords
pub_date
document_type
news_desk
section_name
byline
type_of_material
_id
word_count
score
uri

--4--

https://www.nytimes.com/2018/12/18/briefing/president-xi-kpop-huawei.html
Let us help you start your day.

--5--

https://www.nytimes.com/2018/12/18/briefing/president-xi-kpop-huawei.html
https://www.nytimes.com/2018/02/26/world/asia/xi-jinping-career-highlights.html
https://www.nytimes.com/2018/02/25/world/asia/xi-jinping-china.html
https://www.nytimes.com/2019/01/13/opinion/kim-jong-un-trump-china.html
https://www.nytimes.com/reuters/2019/01/23/technology/23reuters-china-markets-tech.html
https://www.nytimes.com/2018/12/09/opinion/trade-war-truce-trump-china.html
https://www.nytimes.com/2017/11/10/insider/the-last-time-president-xi-took-a-question-from-an-american-correspondent.html
https://www.nytimes.com/aponline/2019/01/21/world/asia/ap-as-china-camb

In [10]:
# save articles about president Xi to a dictionary, with title as key and content as value
mydic = {}
for link in mylink:
    xhtml = urlopen(link)
    xsource = xhtml.read()
    xsoup = BeautifulSoup(xsource, 'html.parser')
    content_raw = xsoup.find_all("div", {"class": "css-u5vfum StoryBodyCompanionColumn"})
    mycontent = []
    for p in content_raw:
        mycontent.append(p.get_text())
    mytitle = xsoup.head.title.get_text()
    mydic[mytitle] = mycontent

### Q1 Tokenize

In [5]:
text = ""
for k,v in mydic.items():
    for p in v:
        text += p
# Clean text
# Reference https://stackoverflow.com/questions/14596884/remove-text-between-and-in-python
text = re.sub("([\[]).*?([\]])", "\g<1>\g<2>", text)
text = text.replace("■","").replace("•","").replace("[","").replace("]","") 

### 1a Use NLTK word_tokenize() to tokenize your text

In [6]:
# NLTK tokenize
tokens_nltk = word_tokenize(text)
text_nltk = nltk.Text(tokens_nltk)

In [9]:
text

''


### 1b Use SpaCy (or a language-specific tokenizer, if not in SpaCy) to tokenize your text. (You may want to refer to the file “Useful SpaCy Commands (Assignment 3).pdf” in Files.) 

In [7]:
# SpaCy tokenize
tokens_spacy = tokenizer(text)

In [8]:
tokens_spacy

### 1c Provide the numbers of tokens for each.

In [9]:
# Number of Tokens using NLTK
num_tk_nltk = len(set(tokens_nltk))
# Number of Tokens using SpaCy
num_tk_spacy = len(set(tokens_spacy))
print(" The numebr of tokens using NLTK tokenizer is", num_tk_nltk, \
      "\n The numebr of tokens using SpaCy tokenizer is", num_tk_spacy)

 The numebr of tokens using NLTK tokenizer is 1818 
 The numebr of tokens using SpaCy tokenizer is 4738


### 1d Provide a few examples of differences (if any). Using small subsets of the text can help you find examples of differences.

In [10]:
tokens_nltk[:1]

['(']

In [11]:
tokens_spacy[:1]

(Want

In [12]:
tokens_nltk[:10]

['(', 'Want', 'to', 'get', 'this', 'briefing', 'by', 'email', '?', 'Here']

In [13]:
tokens_spacy[:10]

(Want to get this briefing by email? Here’s the sign-up.)Good

In [14]:
type(tokens_nltk[:10])

list

In [15]:
type(tokens_spacy[:10])

spacy.tokens.span.Span

Differences:

(1) SpaCy combines punctuations with words. Take the first word of tokens_spacy as an example. The SpaCy tokenizes it to '(Want' tokeni, but NLTK tokenizes to '(' and 'Want'. Because Spacy does not seperate punctuations from words, the number of tokens is larger than the NLTK one.

(2) The format of results are also different. SpaCy tokenizes strings to spacy.tokens.span.Span. NLTK tokenizes strings to a list of strings.


### Q2. Use Regular Expressions (RegEx, python package “re”) to find: You may want to get the file “regular-expressions-cheat-sheet-v2.pdf” from Files to help.

### 2a All words beginning with a– and ending in –ing or –ed.


In [16]:
# Words in nltk tokenized text
nltk2a = set([x for x in tokens_nltk if re.search("^a.*(ing|ed)$", x)])
print(nltk2a)

{'approved', 'answered', 'amazed', 'adding', 'assumed', 'aimed', 'accusing', 'allowing', 'agreed', 'anything', 'asiabriefing', 'announced', 'angered', 'accepting', 'asked', 'aging', 'argued', 'angling', 'added'}


In [17]:
# Words in spacy tokenized text
spacy2a = []
for token in tokens_spacy:
    word = token.text
    if re.search("^a.*(ing|ed)$", word) and word not in spacy2a:
        spacy2a.append(word)
print(spacy2a)

['accusing', 'announced', 'angered', 'anything', 'adding', 'accepting', 'added', 'aging', 'angling', 'allowing', 'amazed', 'approved', 'aimed', 'agreed', 'asked', 'assumed', 'answered']


In [18]:
# Difference of results
print(len(nltk2a))
print(len(spacy2a))

set(nltk2a) - set(spacy2a)

19
17


{'argued', 'asiabriefing'}

### 2b All words with –ing– in the middle of the word.

In [19]:
# Words in nltk tokenized text
nltk2b = set([x for x in tokens_nltk if re.search("^(?!ing).+ing.*(?<!ing)$", x)])
print(nltk2b)
# Check results
nltk2bcheck = set([x for x in tokens_nltk if ("ing" in x and x[:3] != 'ing' and x[-3:] != 'ing')])
# Compare results
nltk2bcheck == nltk2b

{'increasingly', 'dealings', 'document.Washington', 'single', 'seemingly', 'Washington', 'making.China', 'contingency', 'LivingTips', 'Arlington', 'readings', 'mornings', 'rings', 'Jennings', 'things', 'singular', 'grudgingly', 'willingness'}


True

In [20]:
# Words in spacy tokenized text
spacy2b = []
for token in tokens_spacy:
    word = token.text
    if re.search("^(?!ing).+ing.*(?<!ing)$", word) and (word not in spacy2b):
        spacy2b.append(word)
print(spacy2b)
# Check results
spacy2bcheck = []
for token in tokens_spacy:
    word = token.text
    if ('ing' in word) and (word[:3] != 'ing') and (word[-3:] != 'ing') and (word not in spacy2bcheck):
        spacy2bcheck.append(word)
# Compare results
spacy2bcheck == spacy2b

['morning.', 'Jinping’s', 'willingness', 'growing,', 'contingency', 'LivingTips', 'rings?', 'living,”', 'mornings', 'asiabriefing@nytimes.com.HONG', 'Xiaoping,', 'single', 'crying.', 'meeting,', 'making,', 'Beijing’s', 'Arlington,', 'increasingly', 'document.Washington', 'anything,', 'Beijing,', 'singular', 'readings', 'Washington,', 'things', 'marching,', 'winning,', 'Washington', 'King’s', 'Jennings,', 'Jennings', 'Jinping.', 'seemingly', 'building,', 'grudgingly', 'dealings', 'flashing,', 'making.China', 'saying,”', 'Dayspring,', 'Beijing.', 'traveling,', 'expecting,', "Beijing's"]


True

In [21]:
# Difference of results
print(len(nltk2b))
print(len(spacy2b), "\n")

print("Words in the results of SpaCy but not in NLTK's", set(spacy2b) - set(nltk2b),"\n")
print("Words in the results of NLTK but not in SpaCy's", set(nltk2b) - set(spacy2b))

18
44 

Words in the results of SpaCy but not in NLTK's {'morning.', 'rings?', 'Jinping’s', 'living,”', "Beijing's", 'making,', 'expecting,', 'building,', 'Jennings,', 'King’s', 'asiabriefing@nytimes.com.HONG', 'Xiaoping,', 'Arlington,', 'Jinping.', 'Beijing.', 'Beijing’s', 'saying,”', 'traveling,', 'winning,', 'crying.', 'marching,', 'anything,', 'Beijing,', 'meeting,', 'Washington,', 'flashing,', 'growing,', 'Dayspring,'} 

Words in the results of NLTK but not in SpaCy's {'Arlington', 'rings'}


### 2c c. After removing all punctuation, provide a count of all words in your text containing numbers or punctuation and letters (if any).

In [2]:
def remove_punc(l):
    results = []
    for s in l:
        s = re.sub(r'[^\w\s]','',s, re.UNICODE)
        if s:
            results.append(s)
    return results

In [5]:
a = 'dfsdf , dsfsdf, dsf'
remove_punc(a)

['d',
 'f',
 's',
 'd',
 'f',
 ' ',
 ' ',
 'd',
 's',
 'f',
 's',
 'd',
 'f',
 ' ',
 'd',
 's',
 'f']

In [23]:
# Remove punctuation for NLTK text
removedtext_nltk = remove_punc(tokens_nltk)

# Remove punctuation for SpaCy text
tokenlistSpacy = [token.text for token in tokens_spacy]
removedtext_spacy = remove_punc(tokenlistSpacy)

In [24]:
def check_num_punc(l):
    num = []
    punc = []
    for w in l:
        n = re.search('\d+', w)
        p = re.search('[^\w\s]',w)
        if n and w not in num:
            num.append(w)
        if p and p not in punc:
            punc.append(w)
    return num, punc

In [25]:
num_nltk, punc_nltk = check_num_punc(removedtext_nltk)
num_spacy, punc_spacy = check_num_punc(removedtext_spacy)

In [26]:
print('There are', len(num_nltk), 'words containing numbers using NLTK text.')
print('There are', len(punc_nltk), 'words containing punctuation using NLTK text.')
print('There are', len(num_spacy), 'words containing numbers using SpaCy text.')
print('There are', len(punc_nltk), 'words containing punctuation using SpaCy text.')

There are 20 words containing numbers using NLTK text.
There are 0 words containing punctuation using NLTK text.
There are 21 words containing numbers using SpaCy text.
There are 0 words containing punctuation using SpaCy text.


In [27]:
print('Using NLTK text, we get the following words containing numbers:', num_nltk)
print('Using SpaCy text, we get the following words containing numbers:', num_spacy)

Using NLTK text, we get the following words containing numbers: ['40th', '2016', '10000', '240', '58', '2020', '52', '300', '1960s', '2012', '2013', '1', '14', '2024', '90day', '2014', '2009', '12', '2015', '13']
Using SpaCy text, we get the following words containing numbers: ['40th', '2016', '10000', '240', '58', '2020', '52', '300', '1960s', '2012', '2013Controlling', '2013', '1', '14', '2024', '90day', '2014', '2009', '12', '2015', '13']


In [28]:
print("One word is in the results of SpaCy text but not in the results of NLTK text", set(num_spacy)-set(num_nltk))

One word is in the results of SpaCy text but not in the results of NLTK text {'2013Controlling'}


### Q3 Provide a selection of different “stems” from the 3 NLTK stemmers: a. Porter b. Lancaster c. Snowball

In [29]:
def save_stemmer(l,stemmer):
    result = []
    for w in l:
        s = stemmer.stem(w)
        if s not in result and not s.isspace():
            result.append(s)
    return result

In [30]:
# Porter
stemmer1 = PorterStemmer()
stemmer_results_1_nltk = save_stemmer(removedtext_nltk, stemmer1)

In [31]:
# Lancaster
stemmer2 = LancasterStemmer()
stemmer_results_2_nltk = save_stemmer(removedtext_nltk, stemmer2)

In [32]:
# Snowball
stemmer3 = SnowballStemmer("english")
stemmer_results_3_nltk = save_stemmer(removedtext_nltk, stemmer3)

In [33]:
# Words have different stems using various stemmers
different_stemmer_origin_word = []
for w in removedtext_nltk:
    r1 = stemmer1.stem(w)
    r2 = stemmer2.stem(w)
    r3 = stemmer3.stem(w)
    if not (r1 == r2 == r3):
        different_stemmer_origin_word.append([w,r1,r2,r3])

In [34]:
different_stemmer_origin_word

[['this', 'thi', 'thi', 'this'],
 ['Here', 'here', 'her', 'here'],
 ['Xi', 'Xi', 'xi', 'xi'],
 ['defiance', 'defianc', 'defy', 'defianc'],
 ['Huawei', 'huawei', 'huawe', 'huawei'],
 ['corporate', 'corpor', 'corp', 'corpor'],
 ['culture', 'cultur', 'cult', 'cultur'],
 ['Xi', 'Xi', 'xi', 'xi'],
 ['In', 'In', 'in', 'in'],
 ['anniversary', 'anniversari', 'annivers', 'anniversari'],
 ['China', 'china', 'chin', 'china'],
 ['liberalization', 'liber', 'lib', 'liber'],
 ['Xi', 'Xi', 'xi', 'xi'],
 ['delivered', 'deliv', 'del', 'deliv'],
 ['his', 'hi', 'his', 'his'],
 ['Communist', 'communist', 'commun', 'communist'],
 ['Party', 'parti', 'party', 'parti'],
 ['has', 'ha', 'has', 'has'],
 ['solely', 'sole', 'sol', 'sole'],
 ['country', 'countri', 'country', 'countri'],
 ['growth', 'growth', 'grow', 'growth'],
 ['policies', 'polici', 'policy', 'polici'],
 ['have', 'have', 'hav', 'have'],
 ['totally', 'total', 'tot', 'total'],
 ['He', 'He', 'he', 'he'],
 ['only', 'onli', 'on', 'onli'],
 ['obliquely',

### Q4 Provide a count of unique Lemmas in your text using the NLTK WordNet lemmatizer. Provide a count of unique Lemmas in your text using the SpaCy lemmatizer. Provide a few examples where the lemmatizers gave different lemmas (if any), please do also include the original word/token from the text.

### 4a

In [35]:
# Using the NLTK WordNet lemmatizer for puntuation-removed NLTK tokentized text
lemmatizer = WordNetLemmatizer()
lemma_nltk = []
d1 = {}
for i in removedtext_nltk:
    l = lemmatizer.lemmatize(i)
    if l not in lemma_nltk:
        lemma_nltk.append(l)
        d1[l] = i

In [36]:
print("There are", len(lemma_nltk), "unique Lemmas in my text using the NLTK WordNet lemmatizer")

There are 1704 unique Lemmas in my text using the NLTK WordNet lemmatizer


### 4b

In [37]:
lemma_spacy = []
d2 = {}
for token in tokens_spacy:
    s = token.lemma_
    if s not in lemma_spacy:
        lemma_spacy.append(s)
        d2[s] = str(token)

In [38]:
print("There are", len(lemma_spacy), "unique Lemmas in my text using the SpaCy lemmatizer")

There are 1837 unique Lemmas in my text using the SpaCy lemmatizer


### 4c

In [39]:
# Provide 10 examples 
num = 0
for k1, v1 in d1.items():
    for k2, v2 in d2.items():
        if v1 == v2 and k1!=k2:
            num+=1
            if num < 11: 
                print('For word', v1,'; the NLTK lemmatizer lemmatizes it to', k1,\
                      '; the Spacy lemmatizer lemmatizes it to', k2)
            else:
                break

For word briefing ; the NLTK lemmatizer lemmatizes it to briefing ; the Spacy lemmatizer lemmatizes it to brief
For word depicted ; the NLTK lemmatizer lemmatizes it to depicted ; the Spacy lemmatizer lemmatizes it to depict
For word delivered ; the NLTK lemmatizer lemmatizes it to delivered ; the Spacy lemmatizer lemmatizes it to deliver
For word has ; the NLTK lemmatizer lemmatizes it to ha ; the Spacy lemmatizer lemmatizes it to have
For word been ; the NLTK lemmatizer lemmatizes it to been ; the Spacy lemmatizer lemmatizes it to be
For word stunning ; the NLTK lemmatizer lemmatizes it to stunning ; the Spacy lemmatizer lemmatizes it to stun
For word its ; the NLTK lemmatizer lemmatizes it to it ; the Spacy lemmatizer lemmatizes it to its
For word referred ; the NLTK lemmatizer lemmatizes it to referred ; the Spacy lemmatizer lemmatizes it to refer
For word said ; the NLTK lemmatizer lemmatizes it to said ; the Spacy lemmatizer lemmatizes it to say
For word doused ; the NLTK lemmati

### Q5 In 3 to 5 sentences, comment on what you found when using the various tokenizers, stemmers and lemmatizers on your text(s)/corpus – specifically describe similarities and differences you detected.

(1) Spacy tikenizer combines punctuations with words, so the number of tokens using NLTK is quite large. 


(2) The results of using Porter and Snowball stemmers are similar, except that Porter does not change caps to lowercases. 


(3) As for lemmatizers, SpaCy lemmatizer is more accurate. For example, for words: 'referred', 'has' and 'doused', NLTK cannot recognize the root verb but the Spacy lemmatizer could.